# Exploratory Data Analysis on US Consumer Finance Complaints

## Introduction
This notebook performs exploratory data analysis (EDA) on the US Consumer Finance Complaints dataset. The goal is to understand the nature of the complaints and identify any patterns or trends in the data.

## Import Libraries


In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn.cluster
import pyspark
from pyspark.sql import SparkSession



## Load data

In [22]:
# Load the dataset
spark = SparkSession.builder.appName("ConsumerComplaints").getOrCreate()
df = spark.read.csv("../data/consumer_complaints.csv", header=True, inferSchema=True)

#Sampling the data
df = df.sample(0.02)

# Display the first few rows of the dataframe
df.show()
print(f"Number of sampled entries: {len(df)}")

FileNotFoundError: [WinError 2] The system cannot find the file specified

In [ ]:
#Look at the values of each column
df.info()

In [ ]:
df.describe()